In [1]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout

# what follows is from the OpenMM script builder
# with some minor modifications...

## Simulation setup

In [2]:
# read in a starting structure for ethane and the
# corresponding force field file
pdb = app.PDBFile('ethane.pdb')
forcefield = app.ForceField('ethane.gaff2.xml')

# setup system by taking topology from pdb file;
# run gas phase simulation with 2 fs time step (using SHAKE)
# at 323.15 K using a Langevin thermostat (integrator) with
# coupling constant of 5.0 ps^-1
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, 
                                 constraints=app.HBonds)
integrator = mm.LangevinIntegrator(323.15*unit.kelvin, 5.0/unit.picoseconds, 
                                   2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName('CPU')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

## Energy minimization

This reduces the potential energy of the system before beginning dynamics. This eliminates "bad" (i.e., overly close) contacts and generally leads to more stable simulation behavior.

In [3]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy before minimization is %s" % st.getPotentialEnergy())

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy after minimization is %s" % st.getPotentialEnergy())

Minimizing...
Potential energy before minimization is 4.4678182248106335 kJ/mol
Potential energy after minimization is 4.390208155300384 kJ/mol


## Equilibration

We will run a (very) short equilibration simulation to bring the molecule up to our desired temperature.  If this were a periodic system, we would also aim to bring the density/volume to equilibrium at the desired pressure.

In [4]:
print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,13.788899092462412,197.94450691316106
200,26.109636365885002,125.14605931422169
300,24.122440092287718,367.2174533359751
400,15.145827562754254,687.6510731488905
500,31.574085510569407,358.818290171944
600,18.58742232375806,406.1882053832439
700,19.241000440004584,202.85584122717933
800,21.924753107560864,217.6676681204013
900,13.941530818229479,201.263406526695
1000,16.023659750899718,136.86550894747646
1100,11.910021897358511,383.5500291305758
1200,21.660944781546064,392.28220747121907
1300,19.230261162966073,427.270867519712
1400,24.032550735115695,219.9920296965338
1500,17.2253306241009,472.6930373683523
1600,16.314141395115364,421.0192695890868
1700,23.917830742378623,193.20619678813514
1800,15.405252909588937,466.218899213151
1900,30.025727045701082,537.6477941694633
2000,27.14277977658798,429.752619471043
2100,14.43350300739955,279.80976154060045
2200,20.518228489897318,141.98136428454686
2300,13.3995612

## Production

Now we run a long MD simulation with parameters that are identical to the equilibration phase (other than simulation length, of course!).  We will also save a trajectory file (i.e., corodinats vs. time) of this simulation that we can analyze afterward using MDTraj (or other trajectory analysis tools).

In [5]:
print('Running Production...')

simulation.reporters.clear()
# output basic simulation information below every 25000 steps/50 ps
simulation.reporters.append(app.StateDataReporter(stdout, 25000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))
# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 50 steps/0.10 ps
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 50))

# run the simulation for 5x10^5 steps/1 ns
simulation.step(500000)
print('Done!')

Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
25000,50.00000000001514,28.192604140778357,693.9456615986428,0
50000,99.99999999994834,33.51061788361109,448.83142287435004,356
75000,149.99999999998812,18.574409397132566,245.13506287655932,357
100000,200.00000000022686,26.38754745346808,182.16558839139307,361
125000,250.0000000004656,17.128476829766107,346.4429486812472,359
150000,300.00000000070435,26.73613724186292,339.20427975161226,362
175000,350.0000000009431,14.614364588620429,219.76201576703926,358
200000,400.00000000118183,30.591308014520962,484.113518556976,354
225000,450.0000000014206,24.676050098844485,223.8522828349982,353
250000,500.0000000016593,28.232542661985658,259.7295624165671,352
275000,550.000000000818,25.775302588627845,249.75588241347938,351
300000,599.9999999996356,33.94728690321568,198.50021486113164,350
325000,649.9999999984533,27.692031671881264,239.1193787971309,350
350000,699.999999997271,18.850364975